# Project Introduction:
Primer laboratorio de aprendizaje automático, desarrollado por Sergio Barragán Blanco (100472343) y Eduardo Alarcón Navarro (100472175). 
Grupo 17.


# 1. Problema de Clasificación
A continuación, dividiremos la energía entre alta y baja siguiendo las instrucciones (alta si es mayor que el tercer quartil). Y entrenaremos el mejor modelo anterior para ver los resultados, posteriormente elegiremos algunos modelos de Clasificación para entrenar este nuevo problema


In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import PowerTransformer, RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR, SVC
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


También, vamos a importar el documento con los datos de entrenamiento, y le vamos a eliminar las columnas innecesarias

In [2]:

# Load the data
data = pd.read_csv('wind_ava.csv.gz', compression='gzip')

# FIlter the data to only include the columns that end in 13
data = data.filter(regex='13$|energy')
#print(data)

y = data['energy']
x = data.drop(columns=['energy'])


In [ ]:
# test_size vamos a escoger 0.2 ya que tenemos datos en un rango de 5 años en orden (2015-2019) por lo que (asumiendo que el numero de datos para todos los años es similar) usaremos 2019 como test final.
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)
#print(f"Fechas de tain {train.iloc[0].datetime}-{train-iloc[-1].datetime}")

model = SVR(C=701, degree=2, gamma='auto', kernel='rbf', epsilon=0.1)

pipe = Pipeline([
    ('scaler', PowerTransformer()),
    ('model', model)
])

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
# Comprobamos que la RMSE sea igual que en el ejemplo anterior
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

# Ahora vamos a clasificar entre alta y baja, a ver que tan preciso es
tercer_cuartil = np.quantile(y_train, 0.75)

# Asignar etiquetas
y_test_class = np.where(y_test > tercer_cuartil, 1, 0)

# Predecir valores
y_pred_class = np.where(y_pred > tercer_cuartil, 1, 0)

# Evaluar la clasificación
accuracy = accuracy_score(y_test_class, y_pred_class)
precision = precision_score(y_test_class, y_pred_class, pos_label=1)
recall = recall_score(y_test_class, y_pred_class, pos_label=1)
f1_alta = f1_score(y_test_class, y_pred_class, pos_label=1)
f1_baja = f1_score(y_test_class, y_pred_class, pos_label=0)
balanced_accuracy = balanced_accuracy_score(y_test_class, y_pred_class)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", "ALTA = ", f1_alta, "BAJA = ", f1_baja)
print("Balanced Accuracy:", balanced_accuracy)

# Matriz de confusión
confusion_matrix = pd.crosstab(y_test_class, y_pred_class, rownames=['Real'], colnames=['Predicho'])
print(confusion_matrix)

Dado que el resultado más preciso para nuestro modelo ha sido la accuracy, la mantendremos como principal estimación para futuros modelos.

# Modelos que usan clasificación
A continuación transformaremos los datos para que los valores a predecir sean de clasificación. Debido a que el PowerTransformer() no funciona bien para clasificación, en su defecto, usaremos el segundo mejor scaler, el RobustScaler().


In [5]:

# Paso 3: Entrenamiento del modelo
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)

tercer_cuantil = np.quantile(y_train, 0.75)
y_train = np.where(y_train < tercer_cuantil, 0, 1)
y_test = np.where(y_test < tercer_cuantil, 0, 1)

# 1 Alta
# 0 Baja


Primero probaremos con el KNN Clasifier

In [8]:
param_grid = {
    'n_neighbors': range(5, 40, 5), 
    'weights' : ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': range(1, 25, 5),
    'p' : [1, 2]  # distance `= 1 manhattan, p=2 euclidean
}

cv = TimeSeriesSplit()

knn_model = KNeighborsClassifier()

# Búsqueda de hiperparámetros
grid_search = GridSearchCV(knn_model, param_grid, cv=cv, n_jobs=-1, scoring='f1_macro')
without_params = GridSearchCV(knn_model,{}, cv=cv, n_jobs=-1, scoring='f1_macro')

# Construcción del pipeline
pipe = Pipeline([
    ('scaler', RobustScaler()),
    ('model', grid_search)
])
pipe_without_params = Pipeline([
    ('scaler', RobustScaler()),
    ('model', without_params)
])

# Entrenamiento del pipeline
pipe.fit(X_train, y_train)
pipe_without_params.fit(X_train, y_train)

# Mejor modelo y parámetros
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
best_model_w = without_params.best_estimator_
best_params_w = without_params.best_params_

print("Mejor modelo:", best_model, "<--------->", best_model_w)
print("Mejores parámetros:", best_params, "<--------->", best_params_w)
print("Score:", grid_search.best_score_, "<--------->", without_params.best_score_)

"""
Mejor modelo: KNeighborsClassifier(leaf_size=1, n_neighbors=15, p=1, weights='distance') <---------> KNeighborsClassifier()
Mejores parámetros: {'algorithm': 'auto', 'leaf_size': 1, 'n_neighbors': 15, 'p': 1, 'weights': 'distance'} <---------> {}
Score: 0.784670463832614 <---------> 0.7742702702415876
"""

Mejor modelo: KNeighborsClassifier(leaf_size=1, n_neighbors=15, p=1, weights='distance') <---------> KNeighborsClassifier()
Mejores parámetros: {'algorithm': 'auto', 'leaf_size': 1, 'n_neighbors': 15, 'p': 1, 'weights': 'distance'} <---------> {}
Score: 0.784670463832614 <---------> 0.7742702702415876


"\nMejor modelo: KNeighborsClassifier(leaf_size=1, n_neighbors=15, p=1, weights='distance')\nMejores parámetros: {'algorithm': 'auto', 'leaf_size': 1, 'n_neighbors': 15, 'p': 1, 'weights': 'distance'}\nScore: 0.784670463832614\n"

A continuación probaremos con un Decision Tree Classifier

In [9]:
# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'criterion': ['gini', 'entropy'],  # Criterios de desbalanceo para clasificación
    'max_depth': [None, 10, 12, 13, 14, 15, 16, 17],
    'min_samples_split': range(2, 15, 2),
    'min_samples_leaf': range(1, 10, 2),
    'class_weight': ['balanced']  
}

cv = TimeSeriesSplit()

model = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=cv, n_jobs=-1, verbose=1,
                           scoring='f1_macro')  

model2 = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=cv, n_jobs=-1, verbose=1,
                      scoring='f1_macro')
without_params = GridSearchCV(DecisionTreeClassifier(), {}, cv=cv, n_jobs=-1, verbose=1,
                      scoring='f1_macro')

pipe = Pipeline([
    ('scaler', RobustScaler()),
    ('model', model),
    
])
pipe_without_params = Pipeline([
    ('scaler', RobustScaler()),
    ('model', without_params),
    
])

# Entrenamiento del pipeline
pipe.fit(X_train, y_train)
model2.fit(X_train, y_train)
pipe_without_params.fit(X_train, y_train)

pipe_best_params = pipe.named_steps['model'].best_params_
pipe_score = pipe.named_steps['model'].best_score_

pipe_wp_best_params = pipe_without_params.named_steps['model'].best_params_
pipe_wp_score = pipe_without_params.named_steps['model'].best_score_

best_params = model2.best_params_
score = model2.best_score_

print(f"Mejores parámetros: Sin scaler: {best_params} <-----> Con scaler: {pipe_best_params} <-----------> Sin HPO{pipe_wp_best_params}")
print(f"Mejor puntuación: Sin scaler {score} <-----> Con scaler: {pipe_score} <-----------> Sin HPO: {pipe_wp_score}")

"""
Mejores parámetros: Sin scaler: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 12, 'min_samples_leaf': 9, 'min_samples_split': 14} <-----> Con scaler: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 12, 'min_samples_leaf': 9, 'min_samples_split': 6} <-----------> Sin HPO{}
Mejor puntuación: Sin scaler 0.7887936967961423 <-----> Con scaler: 0.7891987832808501 <-----------> Sin HPO: 0.7367893170622747
"""

Fitting 5 folds for each of 560 candidates, totalling 2800 fits
Fitting 5 folds for each of 560 candidates, totalling 2800 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Mejores parámetros: Sin scaler: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 12, 'min_samples_leaf': 9, 'min_samples_split': 14} <-----> Con scaler: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 12, 'min_samples_leaf': 9, 'min_samples_split': 6} <-----------> Sin HPO{}
Mejor puntuación: Sin scaler 0.7887936967961423 <-----> Con scaler: 0.7891987832808501 <-----------> Sin HPO0.7367893170622747


"\nMejores parámetros: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 9, 'min_samples_split': 8}-----{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 9, 'min_samples_split': 12}\nMejor puntuación: 0.7882982356200342-----0.7883933860404928\n"

Y por último, probaremos con un SVM, ya que el mejor modelo para regresión fué su contraparte, esperamos que dé resultados similares o mejores.

In [ ]:
cv = TimeSeriesSplit()

# Instancia del modelo clasificador SVM
model = SVC()

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'C': range(1, 50),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [2, 3, 4, 5],
    'gamma': ['scale', 'auto'],
    'class_weight': ['balanced'] 
}

# Búsqueda de hiperparámetros
grid_search = GridSearchCV(model,
                                 param_grid,
                                 cv=cv, 
                                 n_jobs=-1, 
                                 scoring='f1_macro',  # Métrica de clasificación
                                 )
without_params = GridSearchCV(model, {}, cv=cv, n_jobs=-1, scoring='f1_macro')

# Construcción del pipeline
pipe = Pipeline([
    ('scaler', RobustScaler()),
    ('model', grid_search)
])
pipe_without_params = Pipeline([
    ('scaler', RobustScaler()),
    ('model', without_params)
])

# Entrenamiento del pipeline
pipe.fit(X_train, y_train)
pipe_without_params.fit(X_train, y_train)
# Obtener el mejor modelo y sus parámetros
best_model = pipe.named_steps["model"].best_estimator_
best_params = pipe.named_steps["model"].best_params_
best_score = pipe.named_steps["model"].best_score_

best_model_w = pipe_without_params.named_steps["model"].best_estimator_
best_params_w = pipe_without_params.named_steps["model"].best_params_
best_score_w = pipe_without_params.named_steps["model"].best_score_

print(f"Mejor modelo: Con HPO: {best_model} <-----> Sin HPO: {best_model_w}")
print(f"Mejores parámetros: {best_params} <-----> Sin HPO: {best_params_w}")
print(f"Score: {best_score} <-----> Sin HPO: {best_score_w}")


# Uso de ChatGPT en esta parte:
Para esta parte, como ya poseíamos un amplio conocimento y experiencia de la parte de regresión, no hubo uso de la herramienta. Con la única excepción de intentar resolver el error que daba nan con cualquier scoring debido a que las clases estaban divididas en altas y bajas. No obstante, ChatGPT no solucionó este problema, por lo que se acabó usando la solución de los profesores (cambiar "alta" y "baja" por 0 y 1)